## 5.1 二维卷积层

在卷积层中使用的运算叫做二维互相关运算(cross-correlation)。
> 一个二维输入数组和一个二维核（kernel）数组通过互相关运算输出一个二维数组

代码实现二维卷积的互相关运算：

In [1]:
import torch
import torch.nn as nn

def corr2d(X,K):
    # X为二维数组，K为二维卷积核
    # 卷积核的高和宽
    h,w=K.shape
    # 卷积运算结果
    Y=torch.zeros( (X.shape[0]-h+1,X.shape[1]-w+1) )
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y

In [2]:
# test
X=torch.tensor([
    [0,1,2],
    [3,4,5],
    [6,7,8]
])
# 卷积核
K=torch.tensor([
    [0,1],
    [2,3]
])
# 卷积运算（即互相关运算）
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

上述的互相关运算，可以被用来构建卷积层。

卷积层的模型参数包括了卷积核和标量偏差。

训练模型时，先初始化卷积核参数和偏差，然后开始迭代。

实现一个卷积层，它的前向计算函数就是corr2d函数加偏量。

代码：

In [3]:
# kernel_size是卷积核的大小

class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super(Conv2D,self).__init__()
        # 声明卷积层的参数：卷积核数组和偏差
        self.weight=nn.Parameter(torch.randn(kernel_size))
        self.bias=nn.Parameter(torch.randn(1))
    # 定义前向计算函数
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

In [4]:
# test
# 图像中物体边缘检测

X=torch.ones(6,8)
X[:,2:6]=0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [5]:
# conv kernel

K=torch.tensor([
    [1,-1]
])


In [6]:
# 计算
Y=corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

上述示例中，通过卷积运算，图像边界被很好的表达出来了。

### 学习核数组

步骤：

+ 构造一个卷积层

+ 初始化卷积层参数

+ 开始迭代，使用平方误差比较Y与卷积层的输出

+ 计算梯度，更新权重。

代码：

In [19]:
# 构造卷积核，size=(1,2)
conv2d=Conv2D(kernel_size=(1,2))

# 迭代次数，学习率
epoches=50
lr=0.01

# 开始迭代
for i in range(epoches):
    Y_hat=conv2d(X)
    l=( (Y_hat-Y)**2 ).sum()
    l.backward()

    # 梯度下降
    conv2d.weight.data-=lr*conv2d.weight.grad
    conv2d.bias.data-=lr*conv2d.bias.grad

    # 梯度清0
    conv2d.weight.grad.fill_(0)
    conv2d.bias.data.fill_(0)

    # print epoch,loss
    if (i+1)%10==0:
        print('epocch %d, loss %.3f' % (i+1,l.item()))

epocch 10, loss 0.152
epocch 20, loss 0.012
epocch 30, loss 0.001
epocch 40, loss 0.000
epocch 50, loss 0.000


In [20]:
print('weight:',conv2d.weight.data)
print('bias:',conv2d.bias.data)

weight: tensor([[ 0.9994, -0.9994]])
bias: tensor([0.])


5-1 节小结：

+ 二维卷积层的核心运算是二维互相关运算

+ 卷积核\[-1,1\]可以被用来检测物体边缘

+ 可以通过数据，来学习卷积核(的参数)

## 5-2 填充、步幅

卷积层中，输入为$n_h * n_w$ ，卷积窗口是$k_h * k_w$， 

则卷积输出为$(n_k - k_h +1 )* (n_w - k_w +1)$

所以，卷积层的输出形状是由输入形状和卷积核形状所决定的。

### 填充

指在输入的宽和高的两侧(即数组的外围，围一圈0),填充元素。

填充会使得输入的数组，增加宽和高的值。

如果在高的两侧填充了$p_h$行，在宽的两侧填充了$p_h$行，那么
输出的结果是：$(n_k - k_h +1 +p_h)* (n_w - k_w +1+p_w)$

通常，为了使输出保持与输入相同的大小，填充值$p_h=k_h-1$，$p_w=k_w-1$。

关于填充：

+ 如果$k_h$是奇数，会在高的两侧分别填充$\frac{p_h}{2}$行，在宽的两侧填充$\frac{p_w}{2}$行。

+ 如果$k_h$是偶数，在输入的顶端一侧填充$\lceil \frac{p_h}{2} \rceil$ 行，而在底端一侧填充$\lfloor \frac{p_h}{2} \rfloor$行。宽的两侧填充同理。 

示例：

使用torch.nn.Conv2d(),实现填充和卷积。

In [2]:
import torch
import torch.nn as nn 

In [1]:
# 定义一个函数，来计算卷积层，
# 对输入做填充，使得输出的形状与输入的形状相同

def comp_conv2d(conv2d,X):
    # (1,1)表示批量大小和通道数
    X=X.view( (1,1) + X.shape)
    Y=conv2d(X)
    # 前两维是批量和通道，在那时不关心，所以排除
    return Y.view(Y.shape[2:]) 

In [3]:
# 情况一：卷积核的宽和高是相等的

# 两侧各填充1行或列。
conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=3,padding=1)


In [4]:
# test
X=torch.rand(8,8)
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

In [7]:
# 情况二：卷积核的宽和高是不等的
# (5-1)/2=2,(3-1)/2=1
# 所以需要设置padding=(2,1)

conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=(5,3),padding=(2,1))
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

### 步幅

卷积核在输入数组的左上方开始滑动，每次滑动的行数和列数称为步幅。

如果在输入数组的高上步幅为$s_h$，在输入数组的宽上步幅为$s_w$，则输出的数组的形状为：
$$ 
\lfloor (n_h - k_h + p_h + s_h)/s_h  \rfloor *
\lfloor (n_w - k_w + p_w + s_w)/s_w  \rfloor
$$

如果输入的数组宽和高能被步幅整除，则输出数组的形状为：
$$
(n_h/s_h)*(n_w/s_w)
$$

示例：

使用torch.nn.Conv2d函数。参数stride=2

In [11]:
conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=3,padding=1,stride=2)

# test1
comp_conv2d(conv2d,X).shape

torch.Size([4, 4])

In [12]:
conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=(3,5),padding=(0,1),stride=(3,4))

# test1
comp_conv2d(conv2d,X).shape

torch.Size([2, 2])

### 本节小结

+ 填充可以增加输出的高和宽

+ 步幅可以减少输出的高和宽。
